In [1]:
%%HTML
<style type='text/css'>
    *{
        # background-color:#E3EDCD;
        # background-color:black;
        # color:white;
        
    }
    h1{
        color:#1976d2;
    }
    h2{
        color:#f57c00;
    }
    h3{
        color:#ba37ff;
    }
    h4{
        color:green;
    }
    table{
        border:1px solid black !important;
        border-collapse:collapse !important;
    }
    th{
        background-color:blueviolet !important;
        text-align:center;
        color:white;
    }
    th,td{
        border:0.1px solid black !important;
        transition:0.2s all liner;
        
    }
    td:hover{
        transform:scale(1.1);
        background-color:orange;
        color:blueviolet;
    }
    .raw{
        white-space:pre;
        color:green;
    }
    #imp{
        color:red;
    }
    #ct{
        text-align:center;
    }
</style>

## 3.12 处理时间序列

由于 Pandas 最初是为金融模型而创建的，因此它拥有一些功能非常强
大的日期、时间、带时间索引数据的处理工具。本节将介绍的日期与时
间数据主要包含三类。


- 时间戳表示某个具体的时间点（例如 2015 年 7 月 4 日上午 7
点）。  
- 时间间隔与周期表示开始时间点与结束时间点之间的时间长度，例  
如 2015 年（指的是 2015 年 1 月 1 日至 2015 年 12 月 31 日这段时  
间间隔）。周期通常是指一种特殊形式的时间间隔，每个间隔长度          
相同，彼此之间不会重叠（例如，以 24 小时为周期构成每一      
天）。   
- 时间增量（time delta）或持续时间（duration）表示精确的时间长   
度（例如，某程序运行持续时间 22.56 秒）。  

在本节内容中，我们将介绍 Pandas 中的 3 种日期 / 时间数据类型的具体
用法。由于篇幅有限，后文无法对 Python 或 Pandas 的时间序列工具进
行详细的介绍，仅仅是通过一个宽泛的综述，总结何时应该使用它们。
在开始介绍 Pandas 的时间序列工具之前，我们先简单介绍一下 Python
处理日期与时间数据的工具。在介绍完一些值得深入学习的资源之后，
再通过一些简短的示例来演示 Pandas 处理时间序列数据的方法。

### 3.12.1 Python的日期与时间工具

在 Python 标准库与第三方库中有许多可以表示日期、时间、时间增量
和时间跨度（timespan）的工具。尽管 Pandas 提供的时间序列工具更适
合用来处理数据科学问题，但是了解 Pandas 与 Python 标准库以及第三
方库中的其他时间序列工具之间的关联性将大有裨益。

01. 原生Python的日期与时间工具：datetime与dateutil

Python 基本的日期与时间功能都在标准库的 datetime 模块中。如
果和第三方库 dateutil 模块搭配使用，可以快速实现许多处理日
期与时间的功能。例如，你可以用 datetime 类型创建一个日期：

In [2]:
from datetime import datetime
datetime(year=2024,month=10,day=12)

datetime.datetime(2024, 10, 12, 0, 0)

或者使用 dateutil 模块对各种字符串格式的日期进行正确解析：

In [5]:
from dateutil import parser
date = parser.parse("12th of October, 2024")
date

datetime.datetime(2024, 10, 12, 0, 0)

一旦<span id='imp'>有了 datetime 对象，就可以进行许多操作了</span>，例如打印出这
一天是星期几：


In [7]:
date.strftime('%A')

'Saturday'

在最后一行代码中，为了打印出是星期几，我们使用了一个标准字
符串格式（standard string format）代码 "%A"，你可以在 Python 的
datetime 文档（https://docs.python.org/3/library/datetime.html）
的“strftime”节          
（https://docs.python.org/3/library/datetime.html#strftime-and-strptimebehavior）查看具体信息。关于 dateutil 的其他日期功能可以通        
过 dateutil 的在线文档（http://labix.org/python-dateutil）学习。还有      
一个值得关注的程序包是 pytz（http://pytz.sourceforge.net/），这         
个工具解决了绝大多数时间序列数据都会遇到的难题：时区。      
datetime 和 dateutil 模块在灵活性与易用性方面都表现出色，     
你可以用这些对象及其相应的方法轻松完成你感兴趣的任意操作。     
但如果你处理的时间数据量比较大，那么速度就会比较慢。就像之       
前介绍过的 Python 的原生列表对象没有 NumPy 中已经被编码的数       
值类型数组的性能好一样，Python 的原生日期对象同样也没有    
NumPy 中已经被编码的日期（encoded dates）类型数组的性能好。       

02. 时间类型数组：NumPy的datetime64类型

Python 原生日期格式的性能弱点促使 NumPy 团队为 NumPy 增加了
自己的时间序列类型。 datetime64 类型将日期编码为 64 位整
数，这样可以让日期数组非常紧凑（节省内存）。 datetime64 需
要在设置日期时确定具体的输入类型：

In [8]:
import numpy as np
date = np.array('2024-10-12',dtype=np.datetime64)
date

array('2024-10-12', dtype='datetime64[D]')

但只要有了这个日期格式，就可以进行快速的向量化运算：

In [9]:
date+np.arange(12)

array(['2024-10-12', '2024-10-13', '2024-10-14', '2024-10-15',
       '2024-10-16', '2024-10-17', '2024-10-18', '2024-10-19',
       '2024-10-20', '2024-10-21', '2024-10-22', '2024-10-23'],
      dtype='datetime64[D]')

因为 NumPy 的 datetime64 数组内元素的类型是统一的，所以这
种数组的运算速度会比 Python 的 datetime 对象的运算速度快很
多，尤其是在处理较大数组时（关于向量化运算的内容已经在 2.3
节介绍过）。

datetime64 与 timedelta64 对象的一个共同特点是，它们都是      
在 基本时间单位（fundamental time unit）的基础上建立的。由于       
datetime64 对象是 64 位精度，所以可编码的时间范围可以是基        
本单元的 2         
64 倍。也就是说，datetime64 在时间精度（time     
resolution）与最大时间跨度（maximum time span）之间达成了一         
种平衡。           
比如你想要一个时间纳秒（nanosecond，ns）级的时间精度，那么       
你就可以将时间编码到 0~2    
64 纳秒或 600 年之内，NumPy 会自动判    
断输入时间需要使用的时间单位。例如，下面是一个以天为单位的     
日期：    


In [18]:
np.datetime64?

Init signature: np.datetime64(self, /, *args, **kwargs)
Docstring:     
If created from a 64-bit integer, it represents an offset from
``1970-01-01T00:00:00``.
If created from string, the string can be in ISO 8601 date
or datetime format.

When parsing a string to create a datetime object, if the string contains
a trailing timezone (A 'Z' or a timezone offset), the timezone will be 
dropped and a User Warning is given.

Datetime64 objects should be considered to be UTC and therefore have an 
offset of +0000.

>>> np.datetime64(10, 'Y')
numpy.datetime64('1980')
>>> np.datetime64('1980', 'Y')
numpy.datetime64('1980')
>>> np.datetime64(10, 'D')
numpy.datetime64('1970-01-11')

See :ref:`arrays.datetime` for more information.

:Character code: ``'M'``
File:           d:\virtualenvs\ai\lib\site-packages\numpy\__init__.py
Type:           type
Subclasses:     

In [15]:
np.datetime64('2024-10-12')

np.datetime64('2024-10-12')

而这是一个以分钟为单位的日期：

In [16]:
np.datetime64('2024-10-12 10:27')

np.datetime64('2024-10-12T10:27')

需要注意的是，时区将自动设置为执行代码的操作系统的当地时
区。你可以通过各种格式的代码设置基本时间单位。例如，将时间
单位设置为纳秒：

In [20]:
np.datetime64('2024-10-12 10:29:33.50', 'ns')

np.datetime64('2024-10-12T10:29:33.500000000')

NumPy 的 datetime64 文档
（http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html）总
结了所有支持相对与绝对时间跨度的时间与日期单位格式代码，表
3-6 对此总结如下。

<table>
    <tr>
        <th>代码</th>
        <th>含义</th>
        <th>时间跨度(相对)</th>
        <th>时间跨度(绝对)</th>
    </tr>
    <tr>
        <td>Y</td>
        <td>年（year）</td>
        <td>± 9.2e18 年</td>
        <td>[9.2e18 BC, 9.2e18 AD]</td>
    </tr>
    <tr>
        <td>M</td>
        <td>月（month）</td>
        <td>± 7.6e17 年</td>
        <td>[7.6e17 BC, 7.6e17 AD]</td>
    </tr>
    <tr>
        <td>W</td>
        <td>周（week）</td>
        <td>± 1.7e17 年</td>
        <td>[1.7e17 BC, 1.7e17 AD]</td>
    </tr>
    <tr>
        <td>D</td>
        <td>日（day）</td>
        <td>± 2.5e16 年</td>
        <td>[2.5e16 BC, 2.5e16 AD]</td>
    </tr>
    <tr>
        <td>h</td>
        <td>时（hour）</td>
        <td>± 1.0e15 年</td>
        <td>[1.0e15 BC, 1.0e15 AD]</td>
    </tr>
    <tr>
        <td>m</td>
        <td>分（minute）</td>
        <td>± 1.7e13 年</td>
        <td>[1.7e13 BC, 1.7e13 AD]</td>
    </tr>
    <tr>
        <td>s</td>
        <td>秒（second）</td>
        <td>± 2.9e12 年</td>
        <td>[ 2.9e9 BC, 2.9e9 AD]</td>
    </tr>
    <tr>
        <td>ms</td>
        <td>毫秒（millisecond）</td>
        <td>± 2.9e9 年</td>
        <td>[ 2.9e6 BC, 2.9e6 AD]</td>
    </tr>
    <tr>
        <td>us</td>
        <td>微秒（microsecond）</td>
        <td>± 2.9e6 年</td>
        <td>[290301 BC, 294241 AD]</td>
    </tr>
    <tr>
        <td>ns</td>
        <td>纳秒（nanosecond） </td>
        <td>± 292 年</td>
        <td>[ 1678 AD, 2262 AD]</td>
    </tr>
    <tr>
        <td>ps</td>
        <td>皮秒（picosecond） </td>
        <td>± 106 天</td>
        <td>[ 1969 AD, 1970 AD]</td>
    </tr>
    <tr>
        <td>fs</td>
        <td>飞秒（femtosecond）</td>
        <td>± 2.6 小时</td>
        <td>[ 1969 AD, 1970 AD]</td>
    </tr>
    <tr>
        <td>as</td>
        <td>原秒（attosecond）</td>
        <td>± 9.2 秒</td>
        <td>[ 1969 AD, 1970 AD]</td>
    </tr>
</table>

<p id='ct'>表3-6：日期与时间单位格式代码</p>

对于日常工作中的时间数据类型，默认单位都用纳秒
datetime64[ns]，因为用它来表示时间范围精度可以满足绝大部
分需求。
最后还需要说明一点，虽然 datetime64 弥补了 Python 原生的
datetime 类型的不足，但它缺少了许多 datetime（尤其是
dateutil）原本具备的便捷方法与函数，具体内容请参考 NumPy
的 datetime64 文档
（http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html）。

03. Pandas的日期与时间工具：理想与现实的最佳解决方案

Pandas 所有关于日期与时间的处理方法全部都是通过 Timestamp
对象实现的，它利用 numpy.datetime64 的有效存储和向量化接
口将 datetime 和 dateutil 的易用性有机结合起来。Pandas 通过
一组 Timestamp 对象就可以创建一个可以作为 Series 或
DataFrame 索引的 DatetimeIndex，我们将在后面介绍许多类似
的例子。


例如，可以用 Pandas 的方式演示前面介绍的日期与时间功能。我
们可以灵活处理不同格式的日期与时间字符串，获取某一天是星期
几：


In [21]:
import pandas as pd
date = pd.to_datetime('12th of Oct, 2024')
date

Timestamp('2024-10-12 00:00:00')

In [22]:
date.strftime('%A')

'Saturday'

另外，也可以直接进行 NumPy 类型的向量化运算：

In [23]:
 date + pd.to_timedelta(np.arange(12), 'D')

DatetimeIndex(['2024-10-12', '2024-10-13', '2024-10-14', '2024-10-15',
               '2024-10-16', '2024-10-17', '2024-10-18', '2024-10-19',
               '2024-10-20', '2024-10-21', '2024-10-22', '2024-10-23'],
              dtype='datetime64[ns]', freq=None)

In [24]:
pd.to_timedelta?

Signature:
pd.to_timedelta(
    arg: 'str | int | float | timedelta | list | tuple | range | ArrayLike | Index | Series',
    unit: 'UnitChoices | None' = None,
    errors: 'DateTimeErrorChoices' = 'raise',
) -> 'Timedelta | TimedeltaIndex | Series'
Docstring:
Convert argument to timedelta.

Timedeltas are absolute differences in times, expressed in difference
units (e.g. days, hours, minutes, seconds). This method converts
an argument from a recognized timedelta format / value into
a Timedelta type.

Parameters
----------
arg : str, timedelta, list-like or Series
    The data to be converted to timedelta.

    .. versionchanged:: 2.0
        Strings with units 'M', 'Y' and 'y' do not represent
        unambiguous timedelta values and will raise an exception.

unit : str, optional
    Denotes the unit of the arg for numeric `arg`. Defaults to ``"ns"``.

    Possible values:

    * 'W'
    * 'D' / 'days' / 'day'
    * 'hours' / 'hour' / 'hr' / 'h' / 'H'
    * 'm' / 'minute' / 'min' / 'm

下面将详细介绍 Pandas 用来处理时间序列数据的工具。

### 3.12.2 Pandas时间序列：用时间作索引

Pandas 时间序列工具非常适合用来处理带时间戳的索引数据。例如，
我们可以通过一个时间索引数据创建一个 Series 对象：


In [25]:
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04',
'2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)
data

2014-07-04    0
2014-08-04    1
2015-07-04    2
2015-08-04    3
dtype: int64

有了一个带时间索引的 Series 之后，就能用它来演示之前介绍过的
Series 取值方法，可以直接用日期进行切片取值：

In [26]:
data['2014-07-04':'2015-07-04']

2014-07-04    0
2014-08-04    1
2015-07-04    2
dtype: int64

另外，还有一些仅在此类 Series 上可用的取值操作，例如直接通过年
份切片获取该年的数据：

In [27]:
data['2015']

2015-07-04    2
2015-08-04    3
dtype: int64

下面将介绍一些示例，体现将日期作为索引为运算带来的便利性。在此
之前，让我们仔细看看现有的时间序列数据结构

### 3.12.3 Pandas时间序列数据结构

本节将介绍 Pandas 用来处理时间序列的基础数据类型。  
- 针对时间戳数据，Pandas 提供了 Timestamp 类型。与前面介绍的  
一样，它本质上是 Python 的原生 datetime 类型的替代品，但是  
在性能更好的 numpy.datetime64 类型的基础上创建。对应的索  
引数据结构是 DatetimeIndex。    
- 针对时间周期数据，Pandas 提供了 Period 类型。这是利用  
numpy.datetime64 类型将固定频率的时间间隔进行编码。对应的  
索引数据结构是 PeriodIndex。  
- 针对时间增量或持续时间，Pandas 提供了 Timedelta 类  
型。Timedelta 是一种代替 Python 原生 datetime.timedelta 类  
型的高性能数据结构，同样是基于 numpy.timedelta64 类型。对  
应的索引数据结构是 TimedeltaIndex。  

最基础的日期 / 时间对象是 Timestamp 和 DatetimeIndex。这两种对  
象可以直接使用，最常用的方法是 pd.to_datetime() 函数，它可以  
解析许多日期与时间格式。对 pd.to_datetime() 传递一个日期会返  
回一个 Timestamp 类型，传递一个时间序列会返回一个  
DatetimeIndex 类型：  

In [33]:
 dates = pd.to_datetime([datetime(2024, 10, 12), '13th of Oct, 2024',
                    '2024-Oct-14', '10-15-2024', '20241016'])

In [34]:
dates

DatetimeIndex(['2024-10-12', '2024-10-13', '2024-10-14', '2024-10-15',
               '2024-10-16'],
              dtype='datetime64[ns]', freq=None)

任何 DatetimeIndex 类型都可以通过 to_period()(period译:周期) 方法和一个频率代
码转换成 PeriodIndex 类型。下面用 'D'(day) 将数据转换成单日的时间序
列：

In [38]:
dates.to_period?

Signature: dates.to_period(*args, **kwargs)
Docstring:
Cast to PeriodArray/PeriodIndex at a particular frequency.

Converts DatetimeArray/Index to PeriodArray/PeriodIndex.

Parameters
----------
freq : str or Period, optional
    One of pandas' :ref:`period aliases <timeseries.period_aliases>`
    or an Period object. Will be inferred by default.

Returns
-------
PeriodArray/PeriodIndex

Raises
------
ValueError
    When converting a DatetimeArray/Index with non-regular values,
    so that a frequency cannot be inferred.

See Also
--------
PeriodIndex: Immutable ndarray holding ordinal values.
DatetimeIndex.to_pydatetime: Return DatetimeIndex as object.

Examples
--------
>>> df = pd.DataFrame({"y": [1, 2, 3]},
...                   index=pd.to_datetime(["2000-03-31 00:00:00",
...                                         "2000-05-31 00:00:00",
...                                         "2000-08-31 00:00:00"]))
>>> df.index.to_period("M")
PeriodIndex(['2000-03', '2000-05', '2000-08'],
 

In [35]:
 dates.to_period('D')

PeriodIndex(['2024-10-12', '2024-10-13', '2024-10-14', '2024-10-15',
             '2024-10-16'],
            dtype='period[D]')

当用一个日期减去另一个日期时，返回的结果是 TimedeltaIndex 类
型：

In [36]:
dates - dates[0]

TimedeltaIndex(['0 days', '1 days', '2 days', '3 days', '4 days'], dtype='timedelta64[ns]', freq=None)

有规律的时间序列：pd.date_range()

In [41]:
pd.date_range?

Signature:
pd.date_range(
    start=None,
    end=None,
    periods=None,
    freq=None,
    tz=None,
    normalize: 'bool' = False,
    name: 'Hashable | None' = None,
    inclusive: 'IntervalClosedType' = 'both',
    *,
    unit: 'str | None' = None,
    **kwargs,
) -> 'DatetimeIndex'
Docstring:
Return a fixed frequency DatetimeIndex.

Returns the range of equally spaced time points (where the difference between any
two adjacent points is specified by the given frequency) such that they all
satisfy `start <[=] x <[=] end`, where the first one and the last one are, resp.,
the first and last time points in that range that fall on the boundary of ``freq``
(if given as a frequency string) or that are valid for ``freq`` (if given as a
:class:`pandas.tseries.offsets.DateOffset`). (If exactly one of ``start``,
``end``, or ``freq`` is *not* specified, this missing parameter can be computed
given ``periods``, the number of timesteps in the range. See the note below.)

Parameters
----------
st

为了能更简便地创建有规律的时间序列，Pandas 提供了一些方
法：pd.date_range() 可以处理时间戳、pd.period_range() 可以处
理周期、pd.timedelta_range() 可以处理时间间隔。我们已经介绍
过，Python 的 range() 和 NumPy 的 np.arange() 可以用起点、终点
和步长（可选的）创建一个序列。pd.date_range() 与之类似，通过
开始日期、结束日期和频率代码（同样是可选的）创建一个有规律的日
期序列，默认的频率是天：

In [42]:
pd.date_range(start='2015-07-03', end='2015-07-10')

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06',
               '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10'],
              dtype='datetime64[ns]', freq='D')

此外，日期范围不一定非是开始时间与结束时间，也可以是开始时间与
周期数 periods：

In [43]:
 pd.date_range(start='2024-10-12', periods=8)

DatetimeIndex(['2024-10-12', '2024-10-13', '2024-10-14', '2024-10-15',
               '2024-10-16', '2024-10-17', '2024-10-18', '2024-10-19'],
              dtype='datetime64[ns]', freq='D')

你可以通过 freq 参数改变时间间隔，默认值是 D。例如，可以创建一
个按小时变化的时间戳：

In [46]:
pd.date_range(start='2024-10-12',periods=8,freq='h')

DatetimeIndex(['2024-10-12 00:00:00', '2024-10-12 01:00:00',
               '2024-10-12 02:00:00', '2024-10-12 03:00:00',
               '2024-10-12 04:00:00', '2024-10-12 05:00:00',
               '2024-10-12 06:00:00', '2024-10-12 07:00:00'],
              dtype='datetime64[ns]', freq='h')

如果要创建一个有规律的周期或时间间隔序列，有类似的函数
pd.period_range() 和 pd.timedelta_range()。下面是一个以月为
周期的示例：

In [48]:
pd.period_range(start='2024-10', periods=8, freq='M')

PeriodIndex(['2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03',
             '2025-04', '2025-05'],
            dtype='period[M]')

以及一个以小时递增的序列：

In [54]:
pd.timedelta_range(0, periods=10, freq='h')

TimedeltaIndex(['0 days 00:00:00', '0 days 01:00:00', '0 days 02:00:00',
                '0 days 03:00:00', '0 days 04:00:00', '0 days 05:00:00',
                '0 days 06:00:00', '0 days 07:00:00', '0 days 08:00:00',
                '0 days 09:00:00'],
               dtype='timedelta64[ns]', freq='h')

掌握 Pandas 频率代码是使用所有这些时间序列创建方法的必要条件。
接下来，我们将总结这些代码。

### 3.12.4 时间频率与偏移量

Pandas 时间序列工具的基础是时间频率或偏移量（offset）代码。就像
之前见过的 D（day）和 H（hour）代码，我们可以用这些代码设置任意
需要的时间间隔。表 3-7 总结了主要的频率代码。

<table>
    <tr>
        <th>代码</th>
        <th>描述</th>
        <th>代码</th>
        <th>描述</th>
    </tr>
    <tr>
        <td>D</td>
        <td>天（calendar day，按日历算，含双休日）</td>
        <td>B</td>
        <td>天（business day，仅含工作日）</td>
    </tr>
    <tr>
        <td>W</td>
        <td>周</td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>M</td>
        <td>月末(month end)</td>
        <td>BM</td>
        <td>月末（business month end，仅含工作日）</td>
    </tr>
    <tr>
        <td>Q</td>
        <td>季末（quarter end）</td>
        <td>BQ</td>
        <td>季末（business quarter end，仅含工作日）</td>
    </tr>
    <tr>
        <td>A</td>
        <td>年末（year end） </td>
        <td>BA</td>
        <td>年末（business year end，仅含工作日）</td>
    </tr>
    <tr>
        <td>H(新版本为h)</td>
        <td>小时（hours） </td>
        <td>BH</td>
        <td>小时（business hours，工作时间）</td>
    </tr>
    <tr>
        <td>T(新版本为min)</td>
        <td>分钟（minutes）</td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>S</td>
        <td>秒(second)</td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>L</td>
        <td>毫秒(milliseonds)</td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>U</td>
        <td>微秒（microseconds）</td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>N</td>
        <td>纳秒（nanoseconds）</td>
        <td></td>
        <td></td>
    </tr>
</table>

<p id='ct'>表3-7：Pandas频率代码</p>

月、季、年频率都是具体周期的结束时间（月末、季末、年末），而有
一些以 S（start，开始）为后缀的代码表示日期开始（如表 3-8 所
示）。

<table>
    <tr>
        <th>代码</th>
        <th>频率</th>
    </tr>
    <tr>
        <td>MS</td>
        <td>月初（month start）</td>
    </tr>
    <tr>
        <td>BMS</td>
        <td>月初（business month start，仅含工作日）</td>
    </tr>
    <tr>
        <td>QS</td>
        <td>季初（quarter start）</td>
    </tr>
    <tr>
        <td>BQS </td>
        <td>季初（business quarter start，仅含工作日）</td>
    </tr>
    <tr>
        <td>AS</td>
        <td>年初（year start）</td>
    </tr>
    <tr>
        <td>BAS </td>
        <td>年初（business year start，仅含工作日）</td>
    </tr>
</table>

<p id='ct'>表3-8：带开始索引的频率代码</p>

另外，你可以在频率代码后面加三位月份缩写字母来改变季、年频率的
开始时间。

- Q-JAN、BQ-FEB、QS-MAR、BQS-APR 等。
- A-JAN、BA-FEB、AS-MAR、BAS-APR 等。

同理，也可以在后面加三位星期缩写字母来改变一周的开始时间。

- W-SUN、W-MON、W-TUE、W-WED 等。

在这些代码的基础上，还可以将频率组合起来创建的新的周期。例如，
可以用小时（H）和分钟（T）的组合来实现 2 小时 30 分钟：

In [58]:
pd.timedelta_range(0, periods=9, freq="2h30min")

TimedeltaIndex(['0 days 00:00:00', '0 days 02:30:00', '0 days 05:00:00',
                '0 days 07:30:00', '0 days 10:00:00', '0 days 12:30:00',
                '0 days 15:00:00', '0 days 17:30:00', '0 days 20:00:00'],
               dtype='timedelta64[ns]', freq='150min')

所有这些频率代码都对应 Pandas 时间序列的偏移量，具体内容可以在
pd.tseries.offsets 模块中找到。例如，可以用下面的方法直接创建
一个工作日偏移序列：

In [59]:
from pandas.tseries.offsets import BDay
pd.date_range('2015-07-01', periods=5, freq=BDay()) 
#这里用了时间序列的偏移量,利用了BDay表示只取工作日


DatetimeIndex(['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-06',
               '2015-07-07'],
              dtype='datetime64[ns]', freq='B')

关于时间频率与偏移量的更多内容，请参考 Pandas 在线文档“Date
Offset objects”（http://pandas.pydata.org/pandasdocs/stable/timeseries.html#dateoffset-objects）节

### 3.12.5 重新取样、迁移和窗口

用日期和时间直观地组织与获取数据是 Pandas 时间序列工具最重要的
功能之一。Pandas 不仅支持普通索引功能（合并数据时自动索引对齐、
直观的数据切片和取值方法等），还专为时间序列提供了额外的操作。
下面让我们用一些股票数据来演示这些功能。由于 Pandas 最初是为金
融数据模型服务的，因此可以用它非常方便地获取金融数据。例
如，pandas-datareader 程序包（可以通过 conda install
pandas-datareader 进行安装）知道如何从一些可用的数据源导入金
融数据，包含 Yahoo 财经、Google 财经和其他数据源。下面来导入
Google 的历史股票价格：
